In [ ]:
#this will plot MUR and create a geotiff, but perhaps not an improvement.  We I acutally want is a scalable geotiff.  
#need to talk to doron about what they are doing differently for hycom versus mur & where they get the hycom
#data since that site is opendap terrible connection normally

import gdal
import numpy as np
import xarray as xp

def get_sat_filename(date):
    dir_sat='F:/data/sst/jpl_mur/v4.1/'
    syr, smon, sdym, sjdy = str(date.dt.year.data), str(date.dt.month.data).zfill(2), str(date.dt.day.data).zfill(2), str(date.dt.dayofyear.data).zfill(2)
    sat_filename = dir_sat + syr + '/'+ sjdy + '/' + syr + smon + sdym + '090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc'
    exists = os.path.isfile(sat_filename)
    return sat_filename, exists


In [ ]:
#use opendap to read in MUR SST dataset
today = np.datetime64.today
filename,exists = get_sat_filename(today)
ds = xr.open_dataset(filename)
ds.analysed_sst.plot()
fig_fname='F:/data/sst/jpl_mur/v4.1/test.tiff'
fig.savefig(fig_fname, transparent=True, format='tiff')

In [ ]:
#create geotiff
x_size = 36000
y_size = 18000
num_bands = 4
file_name = fig_fname

data = np.ones((y_size, x_size))
driver = gdal.GetDriverByName('GTiff')
data_set = driver.Create(file_name, x_size, y_size, 
                         gdal.GDT_Float32,
                         options=["TILED=YES","COMPRESS=LZW"])

#for i in range(num_bands):
#    data_set.GetRasterBand(i + 1).WriteArray(data[i])
data_set.GetRasterBand.WriteArray(data)  #not sure what GetRasterBand is doing here, no channel interleaves needed

data_set.BuildOverviews("NEAREST", [2, 4, 8, 16, 32, 64])
data_set = None

driver = gdal.GetDriverByName('GTiff')
data_set2 = driver.CreateCopy(file_name, data_set,
                              options=["COPY_SRC_OVERVIEWS=YES",
                                       "TILED=YES",
                                       "COMPRESS=LZW"])
data_set = None
data_set2 = None